# **GPT-4o Realtime API for Transcribing Audio and Fetching Stock Market Information**

In this tutorial, we’ll create a **real-time stock market analysis application** by combining audio transcription with stock data fetching. Using **GPT-4o**, **Gradio**, **LangChain**, and **ChromaDB**, we'll transcribe spoken stock queries and retrieve relevant financial data. This hands-on guide is implemented in a Google Colab notebook for easy experimentation.

---

### **Key Features**
1. **Audio Transcription:** Use GPT-4o to convert speech into text.
2. **Stock Symbol Extraction:** Parse the transcription to extract stock symbols.
3. **Stock Data Retrieval:** Fetch real-time financial data using `yfinance`.
4. **Interactive Interface:** Enable user interaction through a Gradio-powered interface.

---

### **Prerequisites**
Before diving in, ensure you have:
- A Google Colab account
- An OpenAI API key (GPT-4o access)
- Basic Python knowledge

---

### **Step-by-Step Implementation**

#### **1. Install Required Libraries**
First, install the necessary Python libraries in your Colab environment:

In [ ]:
!pip install openai gradio langchain chromadb pydantic yfinance pandas langchain_openai requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:

#### **2. Import Dependencies**
Import essential modules and configure the OpenAI GPT-4o model:

In [ ]:
import base64
import yfinance as yf
import pandas as pd
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import gradio as gr

In [ ]:
import openai
from google.colab import userdata
import os
# Set your OpenAI API details

OPENAI_API_KEY = userdata.get('openai')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

#### **3. Create a Stock Data Fetching Class**
Define a class to fetch stock data using the `yfinance` library:

In [ ]:
class GetStockInfo(BaseModel):
    """Get real-time stock market information for a given symbol."""
    symbol: str = Field(..., description="The stock symbol, e.g., AAPL, GOOGL")

    def fetch_stock_data(self):
        try:
            stock = yf.Ticker(self.symbol)
            info = stock.fast_info  # Use 'fast_info' for basic details
            current_price = info["last_price"]
            day_high = info["day_high"]
            day_low = info["day_low"]
            volume = info["last_volume"]

            return f"""
            Stock: {self.symbol}
            Current Price: ${current_price:.2f}
            Day High: ${day_high:.2f}
            Day Low: ${day_low:.2f}
            Volume: {volume:,}
            """
        except Exception as e:
            return f"Error fetching stock data: {str(e)}"

#### **4. Configure the GPT-4o Model**
Set up the GPT-4o language model for audio transcription:

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-audio-preview",
    temperature=0
)

def audio_to_text(audio_b64: str) -> str:
    """Convert audio to text using GPT-4o."""
    messages = [
        (
            "human",
            [
                {"type": "text", "text": "Transcribe the following:"},
                {"type": "input_audio", "input_audio": {"data": audio_b64, "format": "wav"}},
            ],
        )
    ]
    output_message = llm.invoke(messages)
    return output_message.content

#### **5. Extract Stock Symbol from Text**
Create a utility function to identify stock symbols in the transcribed text:

In [ ]:
import re

def extract_symbol(transcription: str) -> str:
    """Extract stock symbol from transcription text."""
    import re

    # Dictionary of common company names and their stock ticker symbols
    company_to_symbol = {
        "apple": "AAPL",
        "microsoft": "MSFT",
        "amazon": "AMZN",
        "google": "GOOG",
        "alphabet": "GOOG",
        "tesla": "TSLA",
        "facebook": "META",
        "meta": "META"
    }

    # Check if any company name is mentioned in the transcription
    for company, symbol in company_to_symbol.items():
        if company in transcription.lower():
            return symbol

    # If no company name is found, fall back to matching symbols
    match = re.search(r"(?:of|get|for)\s+([A-Z]+)", transcription, re.IGNORECASE)
    if match:
        return match.group(1).strip().upper()
    return None


#### **6. Process Audio File**
Combine transcription, symbol extraction, and stock data retrieval into a single function:

In [ ]:
def process_audio_file(audio_file):
    with open(audio_file, "rb") as audio_file_obj:
        audio_b64 = base64.b64encode(audio_file_obj.read()).decode("utf-8")

    # Transcribe audio
    transcribed_text = audio_to_text(audio_b64)
    print(f"Transcribed Text: {transcribed_text}")  # Debug transcription

    # Extract stock symbol
    symbol = extract_symbol(transcribed_text)
    print(f"Extracted Symbol: {symbol}")  # Debug stock symbol

    if symbol:
        # Fetch stock data
        stock_tool = GetStockInfo(symbol=symbol)
        stock_result = stock_tool.fetch_stock_data()
        return transcribed_text, symbol, stock_result
    else:
        return transcribed_text, "No valid stock symbol extracted.", "Unable to fetch stock data."

#### **7. Build the Gradio Interface**
Design an intuitive Gradio interface for user interaction:

In [ ]:
interface = gr.Interface(
    fn=process_audio_file,
    inputs=gr.Audio(type="filepath"),
    outputs=["text", "text", "text"],
    title="Audio Transcription and Stock Market Analysis",
    description=(
        "Upload an audio file asking about a stock (e.g., 'Get stock price for AAPL'). "
        "This tool will transcribe the audio, extract the stock symbol, and fetch real-time market data."
    ),
).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://64acab53318bc2bc6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Transcribed Text: I'm sorry, I can't provide real-time information such as the current stock price of Apple. Please check a reliable financial news source or stock market app for the latest information.
Extracted Symbol: AAPL
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://64acab53318bc2bc6c.gradio.live


### **How It Works**
1. **Upload an Audio File:** Users provide an audio file with a stock query, such as *"What is the stock price for AAPL?"*.
2. **Audio Transcription:** The file is converted into text using GPT-4o.
3. **Symbol Extraction:** The app extracts the stock symbol (e.g., `AAPL`) from the transcription.
4. **Data Retrieval:** Real-time stock data is fetched using `yfinance`.
5. **Results Display:** The transcription, extracted symbol, and stock data are displayed.

---

### **Conclusion**
This tutorial showcased how to combine audio transcription, natural language processing, and real-time stock data fetching into a single application. With the powerful integration of **GPT-4o**, **LangChain**, **ChromaDB**, and **Gradio**, you can extend this project to support advanced functionalities like:
- Multi-symbol analysis
- Enhanced error handling
- Financial trend prediction

Start building today and unlock the full potential of AI in real-time applications!